In [1]:
import os
import json
import random
import numpy as np
from tqdm import tqdm
from datetime import datetime

In [2]:
# Config
numSamples = 1000
numVars = 1
seed = 2022 # train: 2021, val: 2022, test: 2023
numPoints = [20,21]
decimals = 2
trainRange = [-1.0,1.0]
testRange = [1.1,3.0]
constantsRange = [-2,2]
template = {'EQ':'', 'Skeleton':'', 'X':[], 'Y':0.0, 'XT':[], 'YT':0.0,}
folder = './Dataset'
os.makedirs(folder, exist_ok=True)
maxSumTerms = 6
maxTermsProb = [0.01,0.19,0.19,0.2, 0.2, 0.21]
maxMulTerms = 2
maxExponents = 6
pSum = 0.9
pMul = 0.2

symbols = {"x":['x{}'.format(i+1) for i in range(numVars)], "const":"C"}
templates = {
    'polynomials': '{}*{}**{}'.format(symbols['const'],'{}','{}'), # identity is part of the polynomials
    #'exponentials': '{}*exp({})**{}'.format(symbols['const'],'{}','{}'),
    'logarithms': '{}*log({})**{}'.format(symbols['const'],'{}','{}'),
    'cos': '{}*cos({})**{}'.format(symbols['const'],'{}','{}'),
    'sin': '{}*sin({})**{}'.format(symbols['const'],'{}','{}'),
    'sqrt': '{}*sqrt({})**{}'.format(symbols['const'],'{}','{}'),
    'power': '{}*{}**{}'.format(symbols['const'],'{}','{}'),
    'identity': '{}*{}'.format(symbols['const'],'{}'),
}
templatesProb = [0.55, 0.05, 0.05, 0.05, 0.05, 0.05, 0.2]

now = datetime.now()
time = now.strftime("%d%m%Y_%H%M%S")
dataPath = folder +'/id{}_nv{}_np{}_trR{}_teR{}_templateBased_t{}.json'.format('{}', numVars, numPoints, 
                                                                 trainRange,
                                                                 testRange, 
                                                                 time)
print(dataPath)

./Dataset/id{}_nv1_np[20, 21]_trR[-1.0, 1.0]_teR[1.1, 3.0]_templateBased_t26052021_214001.json


In [3]:
np.random.seed(seed)
random.seed(seed)
rng = np.random.RandomState(seed)

In [4]:
# add a safe wrapper for numpy math functions
from numpy import *
import numpy as np

def divide(x, y):
  x = np.nan_to_num(x)
  y = np.nan_to_num(y)
  return np.divide(x,y+1e-5)

def sqrt(x):
  x = np.nan_to_num(x)
  return np.sqrt(np.abs(x)) 

# Mean square error
def mse(y, y_hat):
    y_hat = np.reshape(y_hat, [1, -1])[0]
    y_gold = np.reshape(y, [1, -1])[0]
    our_sum = 0
    for i in range(len(y_gold)):
        our_sum += (y_hat[i] - y_gold[i]) ** 2

    return our_sum / len(y_gold)

# Mean square error
def relativeErr(y, y_hat):
    y_hat = np.reshape(y_hat, [1, -1])[0]
    y_gold = np.reshape(y, [1, -1])[0]
    our_sum = 0
    for i in range(len(y_gold)):
        if y_gold[i] < 1: 
            # use regular MSE
            our_sum += (y_hat[i] - y_gold[i]) ** 2
        else:
            # use relative MSE
            our_sum += ((y_hat[i] - y_gold[i])/y_gold[i]) ** 2

    return our_sum / len(y_gold)

In [5]:
import sympy
from sympy import sympify, expand

def simplify_formula(formula_to_simplify, digits=4):
    if len("{}".format(formula_to_simplify)) > 1000:
        return "{}".format(formula_to_simplify)
    orig_form_str = sympify(formula_to_simplify)
    if len("{}".format(orig_form_str)) > 1000:
        return "{}".format(orig_form_str)

    if len("{}".format(orig_form_str)) < 700:
        orig_form_str = expand(orig_form_str)

    rounded = orig_form_str

    traversed = sympy.preorder_traversal(orig_form_str)
    # try:
    #     traversed = timing(orig_form_str) #ft(5, timing, kargs={'x':orig_form_str})
    # except FunctionTimedOut:
    #     print("sympy.preorder_traversal(orig_form_str) could not complete within 5 seconds and was terminated.\n")
    # except Exception as e:
    #     # Handle any exceptions that timing might raise here
    #     print("sympy.preorder_traversal(orig_form_str) was terminated.\n")
    #     return False

    for a in traversed:
        if isinstance(a, sympy.Float):
            if digits is not None:
                if np.abs(a) < 10**(-1*digits):
                    rounded = rounded.subs(a, 0)
                else:
                    rounded = rounded.subs(a, round(a, digits))
            elif np.abs(a) < big_eps:
                rounded = rounded.subs(a, 0)

    return "{}".format(rounded).replace(' ','')

In [6]:
symbols

{'x': ['x1'], 'const': 'C'}

In [7]:
templates

{'polynomials': 'C*{}**{}',
 'logarithms': 'C*log({})**{}',
 'cos': 'C*cos({})**{}',
 'sin': 'C*sin({})**{}',
 'sqrt': 'C*sqrt({})**{}',
 'power': 'C*{}**{}',
 'identity': 'C*{}'}

In [8]:
# Generate the data
fileID = 1
print(dataPath)
numTermsChoices = list(range(maxSumTerms))
keys = list(templates.keys())
for i in tqdm(range(numSamples)):
    numTerms = np.random.choice(numTermsChoices, p=maxTermsProb) # choose the number of terms
    eq = '{}'.format(symbols['const']) # add a constant
    for i in range(numTerms): # for each term, generate an expression
        
        # generate an equation
        if np.random.rand() < pSum: # Summation probability
            term = np.random.choice(len(templates), p=templatesProb) # choose a term
            if keys[term] != 'power': # except power which is designed for expression like t1^t2, all the others have the same template
                exponent = np.random.choice(maxExponents) # choose an exponent
                condition = np.random.rand() < pMul
                if exponent != 0: # if exponent is not zero continue, otherwise ignore this term.
                    # generate the first term
                    if keys[term] == 'identity':
                        t1 = (templates[keys[term]] + '').format(np.random.choice(symbols['x']))
                    else:
                        t1 = (templates[keys[term]] + '').format(np.random.choice(symbols['x']), exponent)
                            
                    if condition: # Multiplication Probability (t1*t2)
                        t1 = t1[:-3] if t1[-3:-1] == "**" else t1 # remove the first term exponent
                        # create the second term
                        term2 = np.random.choice(len(templates), p=templatesProb) # choose a term
                        exponent = np.random.choice(maxExponents) # choose an exponent
                        if exponent != 0: # if exponent is not zero continue, otherwise ignore this term.
                            if keys[term] == 'identity':
                                t2 = (templates[keys[term]] + '').format(np.random.choice(symbols['x'])).strip('C*')
                            else:
                                t2 = (templates[keys[term]] + '').format(np.random.choice(symbols['x']), exponent).strip('C*')
                            t1 += '*' + t2
                    eq += '+' + t1
            else:
                # for the special t1^t2 case
                term = np.random.choice(len(templates), p=templatesProb) # choose a term
                exponent = np.random.choice(maxExponents) # choose an exponent
                if exponent != 0: # if exponent is not zero continue, otherwise ignore this term.
                    # generate the first term
                    if keys[term] == 'identity':
                        t1 = (templates[keys[term]] + '').format(np.random.choice(symbols['x'])).strip('C*')
                    else:
                        t1 = (templates[keys[term]] + '').format(np.random.choice(symbols['x']), exponent).strip('C*')
                
                t1 = t1[:-3] if t1[-3:-1] == "**" else t1 # remove the first term exponent
                term = np.random.choice(len(templates), p=templatesProb) # choose a term
                exponent = np.random.choice(maxExponents) # choose an exponent
                if exponent != 0: # if exponent is not zero continue, otherwise ignore this term.
                    # generate the first term
                    if keys[term] == 'identity':
                        t2 = (templates[keys[term]] + '').format(np.random.choice(symbols['x'])).strip('C*')
                    else:
                        t2 = (templates[keys[term]] + '').format(np.random.choice(symbols['x']), exponent).strip('C*')
                t2 = t2[:-3] if t2[-3:-1] == "**" else t2 # remove the first term exponent
                t = 'C*' + (templates['power']+'').format(t1, t2).strip('C*')
                eq += '+' + t                
        
    # generate data points 
    eq = simplify_formula(eq, decimals) # simplify the equation
    skeletonEqn = eq + '' # copy an equation
    chosenPoints = np.random.randint(numPoints[0],numPoints[1]) # for each equation choose the number of points randomly

    # find all constants in the generated equation, generate a random number based on the given boundry
    constants = [round(random.uniform(constantsRange[0], constantsRange[1]), decimals) for i,x in enumerate(skeletonEqn) if x=='C']            
    eq = skeletonEqn.replace('C','{}').format(*constants) if len(constants)>0 else skeletonEqn

    # for each variable, generate the same number of points (x: (numPoints, numVars))
    X = np.round(rng.uniform(low=trainRange[0], high=trainRange[1], size=(chosenPoints,numVars)), decimals) # generate random points uniformly

    # calculate y based on x
    Y = []
    for point in X:
        tmpEq = eq + '' # copy the string
        for varId in range(numVars):
            tmpEq = tmpEq.replace('x{}'.format(varId+1),str(np.round(point[varId], decimals)))
        try: 
            y = eval(tmpEq)
            if type(y) is np.complex128 or type(y) is np.complex:
                print('Type was complex! Why?: {}'.format(tmpEq))
                y = 0 #abs(err.real)
        except ZeroDivisionError:
            print('Zero Division: {}'.format(tmpEq))
            y = 0
        except OverflowError:
            print('Overflow Error: {}'.format(tmpEq))
            y = 0
        except:
            raise Exception('Err to process this equation: {}, original:{}'.format(tmpEq, skeletonEqn)) 
        Y.append(round(y, decimals))

    # generate xt for the test range
    XT = np.round(rng.uniform(low=testRange[0], high=testRange[1], size=(chosenPoints,numVars)), decimals) # generate random points uniformly

    # calculate yt based on xt
    YT = []
    for point in XT:
        tmpEq = eq + '' # copy the string
        for varId in range(numVars):
            tmpEq = tmpEq.replace('x{}'.format(varId+1),str(point[varId]))
        try: 
            y = eval(tmpEq)
            if type(y) is np.complex128 or type(y) is np.complex:
                print('Type was complex! Why?: {}'.format(tmpEq))
                y = 0 #abs(err.real)
        except ZeroDivisionError:
            print('Zero Division: {}'.format(tmpEq))
            y = 0
        except OverflowError:
            print('Overflow Error: {}'.format(tmpEq))
            y = 0
        except:
            raise Exception('Err to process this equation: {}, original:{}'.format(tmpEq, skeletonEqn)) 
        YT.append(round(y, decimals))

    structure = template.copy() # copy the template

    # hold data in the structure
    structure['X'] = X.tolist()
    structure['Y'] = Y
    structure['Skeleton'] = skeletonEqn
    structure['EQ'] = eq
    structure['XT'] = XT.tolist()
    structure['YT'] = YT
    
    #print(structure['Skeleton'])

    # write to a file
    outputPath = dataPath.format(fileID)
    if os.path.exists(outputPath):
        fileSize = os.path.getsize(outputPath)
        if fileSize > 500000000: # 500 MB
            fileID +=1 

    with open(outputPath, "a", encoding="utf-8") as h:
        json.dump(structure, h, ensure_ascii=False)
        h.write('\n')

  0%|                                                                                       | 0/100000 [00:00<?, ?it/s]C:\Users\vpcom\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in log
  """Entry point for launching an IPython kernel.
  0%|                                                                             | 13/100000 [00:00<12:54, 129.06it/s]C:\Users\vpcom\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in double_scalars
  """Entry point for launching an IPython kernel.
  0%|                                                                             | 42/100000 [00:00<07:31, 221.17it/s]

./Dataset/id{}_nv1_np[20, 21]_trR[-1.0, 1.0]_teR[1.1, 3.0]_templateBased_t26052021_214001.json


  0%|                                                                             | 69/100000 [00:00<07:00, 237.59it/s]C:\Users\vpcom\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:1: RuntimeWarning: divide by zero encountered in log
  """Entry point for launching an IPython kernel.
100%|█████████████████████████████████████████████████████████████████████████| 100000/100000 [05:18<00:00, 313.58it/s]
